In [1]:
import pandas as pd
import glob
import os
import re
from typing import DefaultDict

import pickle
import jsonlines
import numpy as np
import pandas as pd
import plotnine
from sklearn.metrics import cluster
from sklearn.metrics import normalized_mutual_info_score as nmi_score

from collections import defaultdict
import operator
from time import time

import sys
sys.path.insert(0, '../src/d02_intermediate/')
from generate_user_to_hashtag_matrix import TweetVocabVectorizer

import numpy as np
from sklearn.cluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import v_measure_score

In [2]:
data_dir = '../data/02_intermediate/01_group/'
bsc_results = '../data/05_model_output/01_group/'

In [3]:
python_output_files = glob.glob(os.path.join(bsc_results, 'bsc_python*'))

In [4]:
with open(python_output_files[0], 'rb') as f:
    model = pickle.load(f)

with open(os.path.join(data_dir, 'vectorizer_ngram_23.obj'), 'rb') as f:
    vectorizer = pickle.load(f)


/home/hubert/.local/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SpectralCoclustering from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
/home/hubert/.local/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [15]:
with open(os.path.join(data_dir, 'user_count_mat_ngram_23.obj'), 'rb') as f:
    csr = pickle.load(f)

In [5]:
features_names = vectorizer.get_feature_names_out()

In [7]:
features_names.shape[0]

136377560

In [27]:
max(model.row_labels_)

729

In [8]:
def assert_helper(condition, true_msg='Condition Fulfilled', false_msg='Error'):
    assert(condition), false_msg
    print(true_msg)

assert_helper(len(features_names)>0, 'Vocabulary is non-zero. Check OK.')

metoosum = np.sum(['#metoo' in i for i in features_names])
assert_helper(metoosum>0, 'At least #MeToo hashtag is in vocab and has count {}. Check OK.'.format(metoosum))

elements_with_eot_token = np.sum(['eottoken' in i for i in features_names])
assert_helper(elements_with_eot_token==0, 'No eot_tokens found in vocabulary. Check OK.', '{} counts of eot_token found in voabulary! Check NOT OK.'.format(elements_with_eot_token))

Vocabulary is non-zero. Check OK.
At least #MeToo hashtag is in vocab and has count 284920. Check OK.
No eot_tokens found in vocabulary. Check OK.


In [11]:
def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'


format_bytes(features_names.itemsize*features_names.size)

(1.0160920023918152, 'gigabytes')

In [21]:
def bicluster_ncut(cocluster, csr, i):
    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        import sys

        return sys.float_info.max
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    # Note: the following is identical to X[rows[:, np.newaxis],
    # cols].sum() but much faster in scipy <= 0.16
    weight = csr[rows][:, cols].sum()
    # weight = csr[rows[:, np.newaxis],cols].sum()
    cut = csr[row_complement][:, cols].sum() + csr[rows][:, col_complement].sum()
    return cut / weight

In [22]:
bicluster_ncut(model, csr, 1)

2007.2

In [ ]:
def number_normalizer(tokens):
    """Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

# # exclude 'comp.os.ms-windows.misc'
# categories = [
#     "alt.atheism",
#     "comp.graphics",
#     "comp.sys.ibm.pc.hardware",
#     "comp.sys.mac.hardware",
#     "comp.windows.x",
#     "misc.forsale",
#     "rec.autos",
#     "rec.motorcycles",
#     "rec.sport.baseball",
#     "rec.sport.hockey",
#     "sci.crypt",
#     "sci.electronics",
#     "sci.med",
#     "sci.space",
#     "soc.religion.christian",
#     "talk.politics.guns",
#     "talk.politics.mideast",
#     "talk.politics.misc",
#     "talk.religion.misc",
# ]
# newsgroups = fetch_20newsgroups(categories=categories)
# y_true = newsgroups.target

# vectorizer = NumberNormalizingVectorizer(stop_words="english", min_df=5)
# cocluster = SpectralCoclustering(
#     n_clusters=len(categories), svd_method="arpack", random_state=0
# )
# kmeans = MiniBatchKMeans(n_clusters=len(categories), batch_size=20000, random_state=0)

# print("Vectorizing...")
# X = vectorizer.fit_transform(newsgroups.data)

# print("Coclustering...")
# start_time = time()
# cocluster.fit(X)
# y_cocluster = cocluster.row_labels_
# print(
#     "Done in {:.2f}s. V-measure: {:.4f}".format(
#         time() - start_time, v_measure_score(y_cocluster, y_true)
#     )
# )

# print("MiniBatchKMeans...")
# start_time = time()
# y_kmeans = kmeans.fit_predict(X)
# print(
#     "Done in {:.2f}s. V-measure: {:.4f}".format(
#         time() - start_time, v_measure_score(y_kmeans, y_true)
#     )
# )

# feature_names = vectorizer.get_feature_names_out()
# document_names = list(newsgroups.target_names[i] for i in newsgroups.target)





def most_common(d):
    """Items of a defaultdict(int) with the highest values.

    Like Counter.most_common in Python >=2.7.
    """
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)


bicluster_ncuts = list(bicluster_ncut(i) for i in range()))
best_idx = np.argsort(bicluster_ncuts)[:5]

print()
print("Best biclusters:")
print("----------------")
for idx, cluster in enumerate(best_idx):
    n_rows, n_cols = model.get_shape(cluster)
    cluster_docs, cluster_words = model.get_indices(cluster)
    if not len(cluster_docs) or not len(cluster_words):
        continue

    # words
    out_of_cluster_docs = model.row_labels_ != cluster
    out_of_cluster_docs = np.where(out_of_cluster_docs)[0]
    word_col = csr[:, cluster_words]
    word_scores = np.array(
        word_col[cluster_docs, :].sum(axis=0)
        - word_col[out_of_cluster_docs, :].sum(axis=0)
    )
    word_scores = word_scores.ravel()
    important_words = list(
        features_names[cluster_words[i]] for i in word_scores.argsort()[:-11:-1]
    )

    print("bicluster {} : {} documents, {} words".format(idx, n_rows, n_cols))
    print("words        : {}\n".format(", ".join(important_words)))
